<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/onlineprocesssimulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Online simulation of an oil stabilization process
In this notebook we will establish code for using NeqSim for simulation of an oil stabilization process using online data

In [1]:
# Setting up the Colab page to use neqsim
%%capture
!pip install neqsim
import neqsim
from neqsim.thermo.thermoTools import *

# Process flow diagram

In [2]:
inputdata = {
  'feedFlowRateWell': 10,
  'wellPressure': 180.0,
  'wellTemperature':100.0,
  'topsidePressure': 90.0,
  'topsideTemperature': 5.0,
  'firstStagePressure': 75.0, 
  'temperatureOilHeater' : 75.9,
  'secondStagePressure': 8.6,
  'thirdStagePressure': 1.9,
  'firstStageSuctionCoolerTemperature': 25.3, 
  'secondStageSuctionCoolerTemperature': 24.5, 
  'thirdStageSuctionCoolerTemperature':25.0,
  'firstStageExportCoolerTemperature': 25.3, 
  'secondStageExportCoolerTemperature': 24.5
}

In [3]:
from neqsim.thermo import fluid, printFrame
from neqsim.process import compressor, cooler, separator3phase, getProcess, clearProcess, mixer, heater, stream, pump, separator, runProcess, stream, saturator, valve, filters, heatExchanger, simpleTEGAbsorber,distillationColumn, waterStripperColumn, recycle2, setpoint, calculator

clearProcess()

wellFluid = fluid('pr')
wellFluid.addComponent("nitrogen", 0.08)
wellFluid.addComponent("CO2", 3.56)
wellFluid.addComponent("methane", 87.36)
wellFluid.addComponent("ethane", 4.02)
wellFluid.addComponent("propane", 1.54)
wellFluid.addComponent("i-butane", 0.2)
wellFluid.addComponent("n-butane", 0.42)
wellFluid.addComponent("i-pentane", 0.15)
wellFluid.addComponent("n-pentane", 0.20)
		
wellFluid.addTBPfraction("C6", 3.24, 84.99/1000.0, 695.0/1000.0)
wellFluid.addTBPfraction("C7", 1.34, 97.87/1000.0, 718.0/1000.0)
wellFluid.addTBPfraction("C8", 1.33, 111.54/1000.0, 729.0/1000.0)
wellFluid.addTBPfraction("C9", 1.19, 126.1/1000.0, 749.0/1000.0)
wellFluid.addTBPfraction("C10", 1.15, 140.14/1000.0, 760.0/1000.0)
wellFluid.addTBPfraction("C11", 1.69, 175.0/1000.0, 830.0/1000.0)
wellFluid.addTBPfraction("C12", 1.5, 280.0/1000.0, 914.0/1000.0)
wellFluid.addTBPfraction("C13", 2.103, 560.0/1000.0, 980.0/1000.0)

wellFluid.setMixingRule(2)
wellFluid.init(0)

wellFluid.setMolarComposition([0.08, 3.56, 87.36, 4.02, 1.54, 0.2, 0.42, 0.15, 0.2, 1.24, 1.34, 1.33, 1.19, 1.15, 1.69, 1.5, 1.03])

wellStream = stream(wellFluid)
wellStream.setName("dry well stream")
wellStream.setFlowRate(inputdata['feedFlowRateWell'], "MSm3/day")
wellStream.setTemperature(inputdata['wellTemperature'], "C")
wellStream.setPressure(inputdata['wellPressure'], "bara")

saturatedFeedGas = saturator(wellStream)
saturatedFeedGas.setName("water saturator")

waterSaturatedFeedGas = stream(saturatedFeedGas.getOutStream())
waterSaturatedFeedGas.setName("water saturated feed gas")

feedTPsetter = heater(waterSaturatedFeedGas)
feedTPsetter.setName('inletTP')
feedTPsetter.setOutPressure(inputdata['topsidePressure'], "bara")
feedTPsetter.setOutTemperature(inputdata['topsideTemperature'], "C")

chokeValve = valve(feedTPsetter.getOutStream())
chokeValve.setOutletPressure(inputdata['firstStagePressure'], 'bara')

feedToOffshoreProcess = stream(chokeValve.getOutStream())
feedToOffshoreProcess.setName("feed to offshore")

firstStageSeparator = separator3phase(feedToOffshoreProcess)
firstStageSeparator.setName("1st stage separator")

oilHeaterFromFirstStage = heater(firstStageSeparator.getOilOutStream())
oilHeaterFromFirstStage.setName("oil heater second stage")
oilHeaterFromFirstStage.setOutTemperature(inputdata['temperatureOilHeater'],'C')

oilThrotValve = valve(oilHeaterFromFirstStage.getOutStream())
oilThrotValve.setName("valve oil from first stage")
oilThrotValve.setOutletPressure(inputdata['secondStagePressure'])

secondStageSeparator = separator3phase(oilThrotValve.getOutStream())
secondStageSeparator.setName("2nd stage separator")

oilThrotValve2 = valve(secondStageSeparator.getOilOutStream())
oilThrotValve2.setName("valve oil from second stage")
oilThrotValve2.setOutletPressure(inputdata['thirdStagePressure'])

thirdStageSeparator = separator3phase(oilThrotValve2.getOutStream())
thirdStageSeparator.setName("3rd stage separator")

oilThirdStageToSep =  wellStream.clone()
oilThirdStageToSep.setName("resyc oil")
thirdStageSeparator.addStream(oilThirdStageToSep)

stableOil = stream(thirdStageSeparator.getOilOutStream())
stableOil.setName("stable oil")

stableOilPump = pump(stableOil,15.0,"stable oil pump")

firstStageCooler = cooler(thirdStageSeparator.getGasOutStream())
firstStageCooler.setName("1st stage cooler")
firstStageCooler.setOutTemperature(inputdata['firstStageSuctionCoolerTemperature'],'C')

firstStageScrubber = separator(firstStageCooler.getOutStream())
firstStageScrubber.setName("1st stage scrubber")

firstStageCompressor = compressor(firstStageScrubber.getGasOutStream())
firstStageCompressor.setName("1st stage compressor")
firstStageCompressor.setOutletPressure(inputdata['secondStagePressure'])
firstStageCompressor.setIsentropicEfficiency(0.75)

secondStageCooler = cooler(firstStageCompressor.getOutStream())
secondStageCooler.setName("2nd stage cooler")
secondStageCooler.setOutTemperature(inputdata['secondStageSuctionCoolerTemperature'],'C')

secondStageScrubber = separator(secondStageCooler.getOutStream())
secondStageScrubber.setName("2nd stage scrubber")

secondStageCompressor = compressor(secondStageScrubber.getGasOutStream())
secondStageCompressor.setName("2nd stage compressor")
secondStageCompressor.setOutletPressure(inputdata['firstStagePressure'])
secondStageCompressor.setIsentropicEfficiency(0.75)

richGasMixer = mixer("fourth Stage mixer")
richGasMixer.addStream(secondStageCompressor.getOutStream())
richGasMixer.addStream(firstStageSeparator.getGasOutStream())

dewPointControlCooler = cooler(richGasMixer.getOutStream())
dewPointControlCooler.setName("dew point cooler")
dewPointControlCooler.setOutTemperature(inputdata['thirdStageSuctionCoolerTemperature'],'C')

dewPointScrubber = separator(dewPointControlCooler.getOutStream())
dewPointScrubber.setName("dew point scrubber")

lpLiqmixer = mixer("LP liq gas mixer");
lpLiqmixer.addStream(firstStageScrubber.getLiquidOutStream());
lpLiqmixer.addStream(secondStageScrubber.getLiquidOutStream());
lpLiqmixer.addStream(dewPointScrubber.getLiquidOutStream());

lpResycle = recycle2("LP liq resycle")
lpResycle.addStream(lpLiqmixer.getOutStream())
lpResycle.setOutletStream(oilThirdStageToSep)

exportCompressor1 = compressor(dewPointScrubber.getGasOutStream())
exportCompressor1.setName("export 1st stage")
exportCompressor1.setOutletPressure(140.0)
exportCompressor1.setIsentropicEfficiency(0.75)

exportInterstageCooler = cooler(exportCompressor1.getOutStream())
exportInterstageCooler.setName("interstage stage cooler")
exportInterstageCooler.setOutTemperature(inputdata['firstStageExportCoolerTemperature'],'C')

exportCompressor2= compressor(exportInterstageCooler.getOutStream())
exportCompressor2.setName("export 2nd stage")
exportCompressor2.setOutletPressure(200.0)
exportCompressor2.setIsentropicEfficiency(0.75)

exportCooler = cooler(exportCompressor1.getOutStream())
exportCooler.setName("export cooler")
exportCooler.setOutTemperature(inputdata['secondStageExportCoolerTemperature'],'C')

exportGas = stream(exportCooler.getOutStream())
exportGas.setName("export gas")

oilprocess = getProcess()


# Start simulation

In [4]:
oilprocess.run()

In [5]:
TVP = stableOil.TVP(20.0, 'C')
cricondenbar = exportGas.CCB('bara')
powerComp1 = oilprocess.getUnit("1st stage compressor").getPower()/1.0e3
powerComp2 = oilprocess.getUnit("2nd stage compressor").getPower()/1.0e3
powerExpComp1 = oilprocess.getUnit("export 1st stage").getPower()/1.0e3
powerExpComp2 = oilprocess.getUnit("export 2nd stage").getPower()/1.0e3

print('TVP [bara] ', TVP)
print('cricondenbar [bara] ', cricondenbar)
print('recomp power 1 [kW] ', powerComp1)
print('recomp power 2 [kW] ', powerComp2)
print('exp power 1 [kW] ', powerExpComp1)
print('exp power 2 [kW] ', powerExpComp2)

TVP [bara]  1.0021664087084787
cricondenbar [bara]  88.46834793531004
recomp power 1 [kW]  144.1193649852365
recomp power 2 [kW]  185.2044110383442
exp power 1 [kW]  7574.093190900219
exp power 2 [kW]  3916.63781092697


#Set new parameters and run new calculation

New input data can be read from online or from a history data set

In [7]:

inputdata = {
  'feedFlowRateWell': 11.1,
  'wellPressure': 180.0,
  'wellTemperature':100.0,
  'topsidePressure': 91.0,
  'topsideTemperature': 5.0,
  'firstStagePressure': 75.0, 
  'temperatureOilHeater' : 75.9,
  'secondStagePressure': 8.6,
  'thirdStagePressure': 1.9,
  'firstStageSuctionCoolerTemperature': 25.3, 
  'secondStageSuctionCoolerTemperature': 24.5, 
  'thirdStageSuctionCoolerTemperature':25.0,
  'firstStageExportCoolerTemperature': 25.3, 
  'secondStageExportCoolerTemperature': 24.5
}

wellStream.setFlowRate(inputdata['feedFlowRateWell'], "MSm3/day")
wellStream.setTemperature(inputdata['wellTemperature'], "C")
wellStream.setPressure(inputdata['wellPressure'], "bara")
feedTPsetter.setOutPressure(inputdata['topsidePressure'], "bara")
feedTPsetter.setOutTemperature(inputdata['topsideTemperature'], "C")
chokeValve.setOutletPressure(inputdata['firstStagePressure'], 'bara')
oilHeaterFromFirstStage.setOutTemperature(inputdata['temperatureOilHeater'],'C')
oilThrotValve.setOutletPressure(inputdata['secondStagePressure'])
oilThrotValve2.setOutletPressure(inputdata['thirdStagePressure'])
firstStageCooler.setOutTemperature(inputdata['firstStageSuctionCoolerTemperature'],'C')
firstStageCompressor.setOutletPressure(inputdata['secondStagePressure'])
secondStageCooler.setOutTemperature(inputdata['secondStageSuctionCoolerTemperature'],'C')
secondStageCompressor.setOutletPressure(inputdata['firstStagePressure'])
dewPointControlCooler.setOutTemperature(inputdata['thirdStageSuctionCoolerTemperature'],'C')
exportInterstageCooler.setOutTemperature(inputdata['firstStageExportCoolerTemperature'],'C')
exportCooler.setOutTemperature(inputdata['secondStageExportCoolerTemperature'],'C')

In [8]:
oilprocess.run()

In [9]:
TVP = stableOil.TVP(20.0, 'C')
cricondenbar = exportGas.CCB('bara')
powerComp1 = oilprocess.getUnit("1st stage compressor").getPower()/1.0e3
powerComp2 = oilprocess.getUnit("2nd stage compressor").getPower()/1.0e3
powerExpComp1 = oilprocess.getUnit("export 1st stage").getPower()/1.0e3
powerExpComp2 = oilprocess.getUnit("export 2nd stage").getPower()/1.0e3

print('TVP [bara] ', TVP)
print('cricondenbar [bara] ', cricondenbar)
print('recomp power 1 [kW] ', powerComp1)
print('recomp power 2 [kW] ', powerComp2)
print('exp power 1 [kW] ', powerExpComp1)
print('exp power 2 [kW] ', powerExpComp2)

TVP [bara]  1.0023697627523414
cricondenbar [bara]  87.92510683416253
recomp power 1 [kW]  160.0088316766515
recomp power 2 [kW]  205.637281998233
exp power 1 [kW]  8404.711415611837
exp power 2 [kW]  4346.2490884992685


#Live calculations

Read data from a process using TagReader: https://github.com/equinor/tagreader-python

Update input data, and run a simulations.